<a href="https://colab.research.google.com/github/BenUCL/Reef-acoustics-and-AI/blob/main/Code/Feature_extraction_with_pretrained_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connect your Google drive so you can access uploaded data:

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Download the pretrained CNN (VGGish) and required packages. The smoke test code block should return 'Looks good to me!' right at the bottom.

In [1]:
!pip install numpy==1.21.5 resampy==0.2.2 tensorflow==1.15 tf_slim==1.1.0 six==1.15.0 soundfile==0.10.3.post1 llvmlite==0.32.1 

""" As package versions began updating this threw errors on the smoke test. 
For a faster download versions could be removed but this may throw errors. 
As of 17/10/22 it gives the below output, but, the smoketest codeblock passes:

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.16.0 requires gast>=0.3.2, but you have gast 0.2.2 which is incompatible.
kapre 0.3.7 requires tensorflow>=2.0.0, but you have tensorflow 1.15.0 which is incompatible.
Successfully installed gast-0.2.2 keras-applications-1.0.8 llvmlite-0.32.1 numba-0.49.1 numpy-1.21.5 resampy-0.2.2 soundfile-0.10.3.post1 tensorboard-1.15.0 tensorflow-1.15.0 tensorflow-estimator-1.15.1 tf-slim-1.1.0
WARNING: The following packages were previously imported in this runtime:
  [numpy]
You must restart the runtime in order to use newly installed versions. """

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 10.1 MB/s 
     |████████████████████████████████| 323 kB 50.3 MB/s 
     |████████████████████████████████| 412.3 MB 23 kB/s 
     |████████████████████████████████| 352 kB 56.8 MB/s 
     |████████████████████████████████| 20.2 MB 82.8 MB/s 
     |████████████████████████████████| 3.8 MB 40.1 MB/s 
     |████████████████████████████████| 50 kB 6.5 MB/s 
     |████████████████████████████████| 503 kB 60.8 MB/s 
     |████████████████████████████████| 3.5 MB 40.3 MB/s 
     |████████████████████████████████| 3.5 MB 34.9 MB/s 
     |████████████████████████████████| 3.3 MB 49.2 MB/s 
     |████████████████████████████████| 3.3 MB 47.2 MB/s 
     |████████████████████████████████| 3.3 MB 43.3 MB/s 
     |████████████████████████████████| 3.3 MB 47.4 MB/s 
     |████████████████████████████████| 3.4 MB 41.5 MB/s 
     |█████████████████████████

In [2]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 77997, done.
remote: Counting objects: 100% (405/405), done.
remote: Compressing objects: 100% (212/212), done.
remote: Total 77997 (delta 245), reused 322 (delta 191), pack-reused 77592
Receiving objects: 100% (77997/77997), 593.54 MiB | 21.41 MiB/s, done.
Resolving deltas: 100% (55399/55399), done.


In [3]:
%cd models/research/audioset/vggish
# Download data files into same directory as code.
!curl -O https://storage.googleapis.com/audioset/vggish_model.ckpt
!curl -O https://storage.googleapis.com/audioset/vggish_pca_params.npz

/content/models/research/audioset/vggish
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  277M  100  277M    0     0  17.7M      0  0:00:15  0:00:15 --:--:-- 13.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 73020  100 73020    0     0   189k      0 --:--:-- --:--:-- --:--:--  189k


In [4]:
# Should say 'Looks good to me!' at the bottom of output
!python vggish_smoke_test.py


Testing your install of VGGish

Log Mel Spectrogram example:  [[-4.47303259 -4.29463765 -4.14939193 ... -3.97474254 -3.94778045
  -3.78685566]
 [-4.48592983 -4.28831745 -4.13994942 ... -3.98374974 -3.94981089
  -3.79512755]
 [-4.46165595 -4.29335712 -4.14907932 ... -3.96438562 -3.9489109
  -3.78621325]
 ...
 [-4.46165595 -4.29335712 -4.14907932 ... -3.96438562 -3.9489109
  -3.78621325]
 [-4.46165595 -4.29335712 -4.14907932 ... -3.96438562 -3.9489109
  -3.78621325]
 [-4.46165595 -4.29335712 -4.14907932 ... -3.96438562 -3.9489109
  -3.78621325]]
2022-10-17 15:48:34.707787: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-10-17 15:48:34.778905: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-10-17 15:48:34.778983: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c

In [14]:
### Change patsh if you structure folders

# Path to the location where your audio file are stored:
audio_dir = r'/content/drive/MyDrive/Reef soundscapes with AI/audio_dir' 

# Path to folder containing vggish setup files and 'AudiosetAnalysis' downloaded from sarebs supplementary
vggish_files = r'/content/drive/MyDrive/Reef soundscapes with AI/Audioset' 

# Output folder for results:
output_folder_1sec = r'/content/drive/MyDrive/Reef soundscapes with AI/Results/' 
output_folder_1min = r'/content/drive/MyDrive/Reef soundscapes with AI/Results'


In [9]:
import os

#Navigate to the folder containing setup files, including AudiosetAnalysis downloaded from sarebs supplementary
os.chdir(vggish_files) 

#import packages/modules
from AudiosetAnalysis import AudiosetAnalysis
import pandas as pd
import time

# may be some redundant imports
from datetime import datetime
from datetime import timedelta
from datetime import time
from time import strftime
import time

Instructions for updating:
non-resource variables are not supported in the long term


Set up the Audioset analysis. 

RUN THIS BLOCK ONCE PER SESSION, otherwise it will error

In [10]:
an = AudiosetAnalysis()
an.setup()



Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.flatten instead.


This code block extracts pretrained CNN (VGGish) features from each example audio file in your GDrive and saves a timestamped CSV to your results folder in the GDrive.

In [12]:
#select files
all_fs = os.listdir(audio_dir) #list of all files in directory
audio_fs = [f for f in all_fs if '.wav' in f.lower() or '.mp3' in f.lower()] #list of all audio files in dir: .wav or .mps

#initiate empty dataframes to save results
results_df_1sec = pd.DataFrame()
results_df_1min = pd.DataFrame() 

# Feature extraction loop
for f in audio_fs:
    """This loop takes the current filename, rips the timestamp, appends the corresponding length of time being analysed
    additively to a new name, calculates VGGish features from each 0.96s chunk, averages these for each 1min file and saves 
    the results to a csv in your GDrive"""
    path = os.path.join(audio_dir, f)
    print(f) #print file name
    
    #extract timestamp from filename - adjust this if using your own naming convention
    t2 = f.split(".")[1][0:4]
    t1 = f.split(".")[3]
    t = t1+t2+'00'
    recording_start_time = pd.to_datetime(t, format='%y%m%d%H%M%S') 
    slice_time = recording_start_time - timedelta(milliseconds=960)
    mean_slice_time = recording_start_time - timedelta(minutes=1)
    
    #calculate feature values
    results = an.analyse_audio(path)
    
    # Uncomment for 0.96s results:
    # r1sec = results['raw_audioset_feats_960ms']
    # for count, r1sec in enumerate(r1sec):
    #     slice_time = slice_time + timedelta(milliseconds=960)
    #     string_time = slice_time.strftime('%H.%M.%S.%f')[:-4]
    #     result_name = f[:-4] + 'T' + string_time + '.wav'
    #     #result_name = f[:-4]+'T'+str(count+1)+'.wav' #use this line if not using ST timestamped files
    #     results_df_1sec[result_name] = pd.Series(results['raw_audioset_feats_960ms'][count])
    
    #Save 1min results:
    r1min = results['raw_audioset_feats_59520ms']
    for count, r1min in enumerate(r1min):
        #store the timestamp
        mean_slice_time = mean_slice_time + timedelta(minutes=1)
        string_time = mean_slice_time.strftime('%H.%M.%S.%f')[:-4]
        result_name = f[:-4] + 'T' + string_time + '.wav'
        #result_name = f[:-4]+'T'+str(count+1)+'.wav' #use this line if not using ST timestamped files
        results_df_1min[result_name] = pd.Series(results['raw_audioset_feats_59520ms'][count])


#save a timestamped csv with 1min results
now = datetime.now()
time_now = now.strftime("%H.%M.%S")
results_df_1min.to_csv(output_folder_1min + '/pretrained_CNN_features_' + time_now + '.csv')

BaF1.1055H.1678278701.180827.3.35.wav
AudiosetAnalysis: Calculating log mel spectrogram for /content/drive/MyDrive/Reef soundscapes with AI/audio_dir/BaF1.1055H.1678278701.180827.3.35.wav
AudiosetAnalysis: Calculating vggish features /content/drive/MyDrive/Reef soundscapes with AI/audio_dir/BaF1.1055H.1678278701.180827.3.35.wav in batches of 60
BaF3.0915H.1678278701.180829.4.52.wav
AudiosetAnalysis: Calculating log mel spectrogram for /content/drive/MyDrive/Reef soundscapes with AI/audio_dir/BaF3.0915H.1678278701.180829.4.52.wav
AudiosetAnalysis: Calculating vggish features /content/drive/MyDrive/Reef soundscapes with AI/audio_dir/BaF3.0915H.1678278701.180829.4.52.wav in batches of 60
BaF3.0533H.1677983769.180829.3.22.wav
AudiosetAnalysis: Calculating log mel spectrogram for /content/drive/MyDrive/Reef soundscapes with AI/audio_dir/BaF3.0533H.1677983769.180829.3.22.wav
AudiosetAnalysis: Calculating vggish features /content/drive/MyDrive/Reef soundscapes with AI/audio_dir/BaF3.0533H.167

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


SaF2.0534D.1677983769.180828.2.21.wav
AudiosetAnalysis: Calculating log mel spectrogram for /content/drive/MyDrive/Reef soundscapes with AI/audio_dir/SaF2.0534D.1677983769.180828.2.21.wav
AudiosetAnalysis: Calculating vggish features /content/drive/MyDrive/Reef soundscapes with AI/audio_dir/SaF2.0534D.1677983769.180828.2.21.wav in batches of 60
SaF2.1112D.1678278701.180828.3.16.wav
AudiosetAnalysis: Calculating log mel spectrogram for /content/drive/MyDrive/Reef soundscapes with AI/audio_dir/SaF2.1112D.1678278701.180828.3.16.wav
AudiosetAnalysis: Calculating vggish features /content/drive/MyDrive/Reef soundscapes with AI/audio_dir/SaF2.1112D.1678278701.180828.3.16.wav in batches of 60
SaF3.1355D.805322778.180829.1.5.wav
AudiosetAnalysis: Calculating log mel spectrogram for /content/drive/MyDrive/Reef soundscapes with AI/audio_dir/SaF3.1355D.805322778.180829.1.5.wav
AudiosetAnalysis: Calculating vggish features /content/drive/MyDrive/Reef soundscapes with AI/audio_dir/SaF3.1355D.8053227